# Capstone Project - The Battle of Neighborhoods

In [1]:
import numpy as np
import pandas as pd
import json as js
from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
CLIENT_ID = 'LKAGTAZ2HUEEW2R1MNBF0L4E5M301N3KYIL4V0EFVY4BPZY0' # your Foursquare ID
CLIENT_SECRET = 'NYECGC5U4STS1BP2TKTJEKT0SIVGW3OGF11MSVKZQ4PNEAHO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LKAGTAZ2HUEEW2R1MNBF0L4E5M301N3KYIL4V0EFVY4BPZY0
CLIENT_SECRET:NYECGC5U4STS1BP2TKTJEKT0SIVGW3OGF11MSVKZQ4PNEAHO


In [6]:
address = '青岛, CH'
geolocator = Nominatim(user_agent="MO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
limit = 300 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius
target = 'Hotel'
category = '4bf58dd8d48988d17c941735'
coordinate='{}, {}'.format(latitude, longitude)

map_MO = folium.Map(location=[latitude, longitude], zoom_start=12)
map_MO

In [7]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&query={}&ll={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    target,
    coordinate,
    radius, 
    limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=LKAGTAZ2HUEEW2R1MNBF0L4E5M301N3KYIL4V0EFVY4BPZY0&client_secret=NYECGC5U4STS1BP2TKTJEKT0SIVGW3OGF11MSVKZQ4PNEAHO&v=20180605&query=Hotel&ll=36.0638034, 120.3781372&radius=10000&limit=300'

In [8]:
results = requests.get(url).json()

In [9]:
results

{'meta': {'code': 200, 'requestId': '5e98042c1e152c001b4fe857'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Qingdao',
  'headerFullLocation': 'Qingdao',
  'headerLocationGranularity': 'city',
  'query': 'hotel',
  'totalResults': 37,
  'suggestedBounds': {'ne': {'lat': 36.15380349000009,
    'lng': 120.48926569859984},
   'sw': {'lat': 35.97380330999991, 'lng': 120.26700870140016}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c36bf412c8020a10dec8700',
       'name': 'Shangri-La Hotel, Qingdao (Shangri-la Hotel)',
       'location': {'address': '9 Xianggang M. Rd.',
        'lat': 36.06405902645172,
        'lng': 120.37348578394506,
        'labeledLatLngs': [{'label': 'd

In [23]:
hotel = results['response']['groups'][0]['items']
hotel_data = pd.json_normalize(hotel)

In [24]:
# filter columns
columns = ['venue.name', 'venue.location.lat', 'venue.location.lng']
hotel_data =hotel_data.loc[:, columns]
# clean columns
hotel_data.columns = [col.split(".")[-1] for col in hotel_data.columns]
hotel_data.columns = ['Hotel', 'Latitude', 'Longitude']

In [25]:
print(hotel_data.shape)
hotel_data.head()

(37, 3)


,Hotel,Latitude,Longitude
0,"Shangri-La Hotel, Qingdao (Shangri-la Hotel)",36.064059,120.373486
1,InterContinental Qingdao (青岛海尔洲际酒店),36.056359,120.393090
2,The Westin Qingdao (青岛威斯汀酒店),36.063612,120.374476
3,Grand Regency Hotel (麗晶大酒店),36.064764,120.403458
4,Crowne Plaza Qingdao (青岛颐中皇冠假日酒店),36.064119,120.394305


In [26]:
map_hotel = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for hotel, lat, lng in zip(hotel_data['Hotel'], hotel_data['Latitude'], hotel_data['Longitude']):
    label = '{}'.format(hotel)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_hotel)  
    
map_hotel

In [31]:
def getNearbyVenues(hotel,latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    
    for hotel, lat, lng in zip(hotel, latitudes, longitudes):
        print(hotel)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            hotel, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Hotel',           
                  'Hotel Latitude', 
                  'Hotel Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
hotel_venues = getNearbyVenues(hotel=hotel_data['Hotel'],
                                latitudes=hotel_data['Latitude'],
                                longitudes=hotel_data['Longitude']
                                  )

Shangri-La Hotel, Qingdao (Shangri-la Hotel)
InterContinental Qingdao (青岛海尔洲际酒店)
The Westin Qingdao (青岛威斯汀酒店)
Grand Regency Hotel (麗晶大酒店)
Crowne Plaza Qingdao (青岛颐中皇冠假日酒店)
Sea View Garden Hotel Qingdao
Hyatt Regency Qingdao (青岛鲁商凯悦酒店)
Holiday Inn
China Community Art and Culture Hotel
Le Méridien Qingdao (青岛万达艾美酒店)
Himalayas Qingdao Hotel
Café Yum
Housing International Hotel
湛山花园酒店 Zhanshan Garden Hotel
阿兹凯乐西餐厅 Rats Keller western restaurant
青岛饭店 Qingdao Hotel
Copthorne Hotel Qingdao (青岛国顿大酒店)
中航翔通游艇会
Sunny World Hotel
Ibis hotel
Haiqing Hotel Qingdao
Four Points by Sheraton Qingdao, Jiaonan
Le Meridien Qingdao Le Ballroom 3
Le Méridien Qingdao West Coast Resort
海景花园大酒店
武胜关度假酒店
Ibis Hotel 宜必思酒店- 青岛
怡堡酒店 The Castle Hotel
汇泉国际大酒店 Huiquan Dynasty Hotel
Huanghai Hotel
Best Western Premier Qingdao Kilin Crown Hotel 青岛麒麟皇冠大酒店
Qingdao Ocean Hotel 青岛远洋大酒店
Sophia International Seaview Hotel Qingdao
青岛奥林匹克帆船中心1号门
青岛啤酒街 | Beer Street
青岛凯越青年旅社 Qingdao Kaiyue Youth Hostel
青岛奥博维特国际青年旅舍 YHA Old Observ

In [33]:
print(hotel_venues.shape)
hotel_venues.head()

(291, 7)


,Hotel,Hotel Latitude,Hotel Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Shangri-La Hotel, Qingdao (Shangri-la Hotel)",36.064059,120.373486,"Shangri-La Hotel, Qingdao (Shangri-la Hotel)",36.064059,120.373486,Hotel
1,"Shangri-La Hotel, Qingdao (Shangri-la Hotel)",36.064059,120.373486,Café Yum,36.064554,120.372957,Café
2,"Shangri-La Hotel, Qingdao (Shangri-la Hotel)",36.064059,120.373486,the mixC (万象城),36.066887,120.373610,Shopping Mall
3,"Shangri-La Hotel, Qingdao (Shangri-la Hotel)",36.064059,120.373486,Q Bar,36.063915,120.373882,Bar
4,"Shangri-La Hotel, Qingdao (Shangri-la Hotel)",36.064059,120.373486,Starbucks (星巴克),36.064684,120.376046,Coffee Shop


In [35]:
hotel_venues.drop(hotel_venues[hotel_venues['Hotel']==hotel_venues['Venue']].index, inplace=True)
hotel_venues.drop(hotel_venues[hotel_venues['Venue Category']=='Hotel'].index, inplace=True)
hotel_venues.drop(hotel_venues[hotel_venues['Venue Category']=='Hostel'].index, inplace=True)
print(hotel_venues.shape)
hotel_venues.head(5)

(225, 7)


,Hotel,Hotel Latitude,Hotel Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,"Shangri-La Hotel, Qingdao (Shangri-la Hotel)",36.064059,120.373486,Café Yum,36.064554,120.372957,Café
2,"Shangri-La Hotel, Qingdao (Shangri-la Hotel)",36.064059,120.373486,the mixC (万象城),36.066887,120.373610,Shopping Mall
3,"Shangri-La Hotel, Qingdao (Shangri-la Hotel)",36.064059,120.373486,Q Bar,36.063915,120.373882,Bar
4,"Shangri-La Hotel, Qingdao (Shangri-la Hotel)",36.064059,120.373486,Starbucks (星巴克),36.064684,120.376046,Coffee Shop
5,"Shangri-La Hotel, Qingdao (Shangri-la Hotel)",36.064059,120.373486,全聚德,36.063293,120.378544,Chinese Restaurant


In [36]:
print('There are {} uniques categories.'.format(len(hotel_venues['Venue Category'].unique())))

There are 49 uniques categories.


In [37]:
hotel_data = hotel_venues.groupby(['Hotel', 'Hotel Latitude', 'Hotel Longitude'])['Venue'].apply(list).apply(lambda x:",".join(x)).to_frame().reset_index()
hotel_data.head()

,Hotel,Hotel Latitude,Hotel Longitude,Venue
0,Best Western Premier Qingdao Kilin Crown Hotel...,36.097424,120.464783,"Lion Mall (金狮广场),青岛国际啤酒城 Qingdao International..."
1,Café Yum,36.064554,120.372957,"the mixC (万象城),Q Bar,Starbucks (星巴克),Pizza Hut..."
2,China Community Art and Culture Hotel,36.072380,120.390326,"船歌鱼水饺,Kona Coffee,Blue Harbor Seafood (蓝港海鲜厨房)..."
3,Copthorne Hotel Qingdao (青岛国顿大酒店),36.054153,120.361848,"第三海水浴场 No.3 Bathing Beach,景福宫,开海红岛海鲜水饺"
4,Crowne Plaza Qingdao (青岛颐中皇冠假日酒店),36.064119,120.394305,"istanbul (土耳其烤肉),Coffee Hunter,Kona Coffee,Sta..."


In [38]:
# one hot encoding
hotel_onehot = pd.get_dummies(hotel_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hotel_onehot['Hotel'] = hotel_venues['Hotel'] 

# move neighborhood column to the first column
fixed_columns = [hotel_onehot.columns[-1]] + list(hotel_onehot.columns[:-1])
hotel_onehot = hotel_onehot[fixed_columns]

hotel_onehot.head()

,Hotel,American Restaurant,Arcade,BBQ Joint,Bar,Beach,Beer Garden,Bookstore,Brewery,Buffet,Café,Chinese Restaurant,Church,Clothing Store,Coffee Shop,Convenience Store,Diner,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Garden,German Restaurant,Grocery Store,Harbor / Marina,Hotel Bar,Hotpot Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Market,Men's Store,Metro Station,Museum,Nightclub,Park,Pizza Place,Plaza,Ramen Restaurant,Resort,Restaurant,Sandwich Place,Seafood Restaurant,Shandong Restaurant,Shopping Mall,Spa,Supermarket,Szechuan Restaurant,Thai Restaurant,Turkish Restaurant,Zhejiang Restaurant
1,"Shangri-La Hotel, Qingdao (Shangri-la Hotel)",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Shangri-La Hotel, Qingdao (Shangri-la Hotel)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,"Shangri-La Hotel, Qingdao (Shangri-la Hotel)",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Shangri-La Hotel, Qingdao (Shangri-la Hotel)",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,"Shangri-La Hotel, Qingdao (Shangri-la Hotel)",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
hotel_grouped = hotel_onehot.groupby('Hotel').mean().reset_index()
hotel_grouped.head()

,Hotel,American Restaurant,Arcade,BBQ Joint,Bar,Beach,Beer Garden,Bookstore,Brewery,Buffet,Café,Chinese Restaurant,Church,Clothing Store,Coffee Shop,Convenience Store,Diner,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Garden,German Restaurant,Grocery Store,Harbor / Marina,Hotel Bar,Hotpot Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Market,Men's Store,Metro Station,Museum,Nightclub,Park,Pizza Place,Plaza,Ramen Restaurant,Resort,Restaurant,Sandwich Place,Seafood Restaurant,Shandong Restaurant,Shopping Mall,Spa,Supermarket,Szechuan Restaurant,Thai Restaurant,Turkish Restaurant,Zhejiang Restaurant
0,Best Western Premier Qingdao Kilin Crown Hotel...,0.000000,0.333333,0.0,0.000000,0.333333,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.000000,0.000000,0.0
1,Café Yum,0.000000,0.000000,0.0,0.100000,0.000000,0.0,0.100000,0.0,0.1,0.000000,0.100000,0.0,0.0,0.100000,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000,0.0,0.0,0.0,0.000000,0.000000,0.1
2,China Community Art and Culture Hotel,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.200000,0.000000,0.0,0.0,0.200000,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
3,Copthorne Hotel Qingdao (青岛国顿大酒店),0.000000,0.000000,0.0,0.000000,0.333333,0.0,0.000000,0.0,0.0,0.000000,0.333333,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0
4,Crowne Plaza Qingdao (青岛颐中皇冠假日酒店),0.071429,0.000000,0.0,0.071429,0.000000,0.0,0.071429,0.0,0.0,0.071429,0.000000,0.0,0.0,0.285714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.214286,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.071429,0.0,0.0,0.0,0.071429,0.071429,0.0


In [40]:
num_top_venues = 5

for hood in hotel_grouped['Hotel']:
    print("----"+hood+"----")
    temp = hotel_grouped[hotel_grouped['Hotel'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Best Western Premier Qingdao Kilin Crown Hotel 青岛麒麟皇冠大酒店----
               venue  freq
0             Arcade  0.33
1              Beach  0.33
2      Shopping Mall  0.33
3   Ramen Restaurant  0.00
4  Korean Restaurant  0.00


----Café Yum----
                  venue  freq
0   Zhejiang Restaurant   0.1
1                Buffet   0.1
2           Pizza Place   0.1
3  Fast Food Restaurant   0.1
4           Coffee Shop   0.1


----China Community Art and Culture Hotel----
                 venue  freq
0          Coffee Shop   0.2
1                 Café   0.2
2  Dumpling Restaurant   0.2
3   Seafood Restaurant   0.2
4   Italian Restaurant   0.2


----Copthorne Hotel Qingdao (青岛国顿大酒店)----
                venue  freq
0   Korean Restaurant  0.33
1               Beach  0.33
2  Chinese Restaurant  0.33
3              Resort  0.00
4              Market  0.00


----Crowne Plaza Qingdao (青岛颐中皇冠假日酒店)----
                venue  freq
0         Coffee Shop  0.29
1  Italian Restaurant  0.21
2           

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [108]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Hotel']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
hotel_venues_sorted = pd.DataFrame(columns=columns)
hotel_venues_sorted['Hotel'] = hotel_grouped['Hotel']

for ind in np.arange(hotel_grouped.shape[0]):
    hotel_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hotel_grouped.iloc[ind, :], num_top_venues)

hotel_venues_sorted.head()

,Hotel,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Best Western Premier Qingdao Kilin Crown Hotel...,Arcade,Beach,Shopping Mall,Zhejiang Restaurant,Clothing Store
1,Café Yum,Zhejiang Restaurant,Coffee Shop,Bar,Fast Food Restaurant,Shopping Mall
2,China Community Art and Culture Hotel,Italian Restaurant,Dumpling Restaurant,Seafood Restaurant,Café,Coffee Shop
3,Copthorne Hotel Qingdao (青岛国顿大酒店),Korean Restaurant,Beach,Chinese Restaurant,Zhejiang Restaurant,Coffee Shop
4,Crowne Plaza Qingdao (青岛颐中皇冠假日酒店),Coffee Shop,Italian Restaurant,American Restaurant,Thai Restaurant,Turkish Restaurant


In [109]:
# set number of clusters
kclusters = 4

hotel_grouped_clustering = hotel_grouped.drop('Hotel', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hotel_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 0, 1, 0, 0, 0, 1, 0, 0], dtype=int32)

In [110]:
# add clustering labels
hotel_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)

hotel_merged = hotel_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
hotel_merged = hotel_merged.join(hotel_venues_sorted.set_index('Hotel'), on='Hotel')

hotel_merged = hotel_merged.dropna()


hotel_merged.head() # check the last columns!

,Hotel,Hotel Latitude,Hotel Longitude,Venue,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Best Western Premier Qingdao Kilin Crown Hotel...,36.097424,120.464783,"Lion Mall (金狮广场),青岛国际啤酒城 Qingdao International...",1,Arcade,Beach,Shopping Mall,Zhejiang Restaurant,Clothing Store
1,Café Yum,36.064554,120.372957,"the mixC (万象城),Q Bar,Starbucks (星巴克),Pizza Hut...",1,Zhejiang Restaurant,Coffee Shop,Bar,Fast Food Restaurant,Shopping Mall
2,China Community Art and Culture Hotel,36.072380,120.390326,"船歌鱼水饺,Kona Coffee,Blue Harbor Seafood (蓝港海鲜厨房)...",0,Italian Restaurant,Dumpling Restaurant,Seafood Restaurant,Café,Coffee Shop
3,Copthorne Hotel Qingdao (青岛国顿大酒店),36.054153,120.361848,"第三海水浴场 No.3 Bathing Beach,景福宫,开海红岛海鲜水饺",1,Korean Restaurant,Beach,Chinese Restaurant,Zhejiang Restaurant,Coffee Shop
4,Crowne Plaza Qingdao (青岛颐中皇冠假日酒店),36.064119,120.394305,"istanbul (土耳其烤肉),Coffee Hunter,Kona Coffee,Sta...",0,Coffee Shop,Italian Restaurant,American Restaurant,Thai Restaurant,Turkish Restaurant


In [111]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hotel_merged['Hotel Latitude'], hotel_merged['Hotel Longitude'], hotel_merged['Hotel'], hotel_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [112]:
Cluster1 = pd.DataFrame(hotel_merged.loc[hotel_merged['ClusterLabels'] ==0, hotel_merged.columns[[0] + list(range(5, hotel_merged.shape[1]))]])
Cluster1

,Hotel,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,China Community Art and Culture Hotel,Italian Restaurant,Dumpling Restaurant,Seafood Restaurant,Café,Coffee Shop
4,Crowne Plaza Qingdao (青岛颐中皇冠假日酒店),Coffee Shop,Italian Restaurant,American Restaurant,Thai Restaurant,Turkish Restaurant
5,"Four Points by Sheraton Qingdao, Jiaonan",Coffee Shop,Hotel Bar,Shopping Mall,Seafood Restaurant,Restaurant
6,Grand Regency Hotel (麗晶大酒店),Japanese Restaurant,Nightclub,Coffee Shop,Zhejiang Restaurant,Clothing Store
8,Himalayas Qingdao Hotel,Shopping Mall,Market,Sandwich Place,Coffee Shop,Zhejiang Restaurant
9,Holiday Inn,Fast Food Restaurant,Electronics Store,Shopping Mall,Seafood Restaurant,Diner
15,InterContinental Qingdao (青岛海尔洲际酒店),Shopping Mall,Coffee Shop,Szechuan Restaurant,Men's Store,Diner
16,Le Meridien Qingdao Le Ballroom 3,Coffee Shop,Hotel Bar,Supermarket,Shopping Mall,Seafood Restaurant
17,Le Méridien Qingdao (青岛万达艾美酒店),Coffee Shop,Hotel Bar,Supermarket,Shopping Mall,Seafood Restaurant
18,Le Méridien Qingdao West Coast Resort,Shopping Mall,Coffee Shop,Chinese Restaurant,Japanese Restaurant,Restaurant


In [113]:
Cluster2 = pd.DataFrame(hotel_merged.loc[hotel_merged['ClusterLabels'] ==1, hotel_merged.columns[[0] + list(range(5, hotel_merged.shape[1]))]])
Cluster2

,Hotel,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Best Western Premier Qingdao Kilin Crown Hotel...,Arcade,Beach,Shopping Mall,Zhejiang Restaurant,Clothing Store
1,Café Yum,Zhejiang Restaurant,Coffee Shop,Bar,Fast Food Restaurant,Shopping Mall
3,Copthorne Hotel Qingdao (青岛国顿大酒店),Korean Restaurant,Beach,Chinese Restaurant,Zhejiang Restaurant,Coffee Shop
7,Haiqing Hotel Qingdao,Chinese Restaurant,Italian Restaurant,Bar,Beach,Seafood Restaurant
10,Housing International Hotel,Chinese Restaurant,Fast Food Restaurant,Buffet,Electronics Store,Korean Restaurant
11,Huanghai Hotel,Beach,Metro Station,Café,Zhejiang Restaurant,Coffee Shop
12,Hyatt Regency Qingdao (青岛鲁商凯悦酒店),Italian Restaurant,Spa,Korean Restaurant,Shandong Restaurant,Zhejiang Restaurant
13,Ibis Hotel 宜必思酒店- 青岛,Hotpot Restaurant,Spa,Church,Grocery Store,German Restaurant
14,Ibis hotel,Grocery Store,Fast Food Restaurant,Zhejiang Restaurant,Hotel Bar,German Restaurant
19,Qingdao Ocean Hotel 青岛远洋大酒店,Grocery Store,Arcade,Fast Food Restaurant,Shopping Mall,Metro Station


In [114]:
Cluster3 = pd.DataFrame(hotel_merged.loc[hotel_merged['ClusterLabels'] ==2, hotel_merged.columns[[0] + list(range(5, hotel_merged.shape[1]))]])
Cluster3

,Hotel,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
20,Sea View Garden Hotel Qingdao,Beach,Zhejiang Restaurant,Clothing Store,Grocery Store,German Restaurant
27,武胜关度假酒店,Plaza,Beach,Zhejiang Restaurant,Grocery Store,German Restaurant
28,汇泉国际大酒店 Huiquan Dynasty Hotel,Plaza,Beach,Zhejiang Restaurant,Grocery Store,German Restaurant
29,海景花园大酒店,Beach,Zhejiang Restaurant,Clothing Store,Grocery Store,German Restaurant


In [115]:
Cluster4 = pd.DataFrame(hotel_merged.loc[hotel_merged['ClusterLabels'] ==3, hotel_merged.columns[[0] + list(range(5, hotel_merged.shape[1]))]])
Cluster4

,Hotel,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
34,"青岛奥博维特国际青年旅舍 YHA Old Observatory, Qingdao",Market,Zhejiang Restaurant,Clothing Store,Grocery Store,German Restaurant


In [116]:
Cluster5 = pd.DataFrame(hotel_merged.loc[hotel_merged['ClusterLabels'] ==4, hotel_merged.columns[[0] + list(range(5, hotel_merged.shape[1]))]])
Cluster5

,Hotel,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
